# Part 9 -- Predictive Modeling

Prepare data for **Singular Value Decomposition (SVD)**.

### Load lib codes:

In [53]:
!pwd

/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter


In [54]:
from os import chdir
chdir('/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/')

from lib import *
from lib.twitter_keys import my_keys
# suppress_warnings()
%matplotlib inline

# DO IT ON RAW DATA --> TFIDF TO LOGISTIC REGRESSION --> GRIDSEARCH TUNE LOG REG & TRY RF, SVC, KNN.

* THEN do a GridSearch on the best one
* THEN CIRCLE BCAK AND EDA ON CLUSTERS

## Logistic Regression

In [71]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

In [61]:
X = joblib.load('../predicting_stock_market_trends_with_Twitter/data/X_tfidf.pickle')

In [62]:
y = joblib.load('../predicting_stock_market_trends_with_Twitter/data/y_AAPL_le.pickle')

In [63]:
X.shape

(68006, 5268)

In [27]:
y.shape

(68006,)

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42)

**Regularization: The problem of overfitting**

You don't want your hypothesis to have high bias (underfit) or take too many features and the learned hypthesis will learn the training set really well, but not generalize to new data as well (predict prices on new data). 

If you think overfitting is occurring, you can REGULARIZATION. Keep all features, but reduce the magnitude. This method works well when you have LOTS of features that contribute a little bit to the value of y, so you might not want to throw them away. Regularization (LASSO) 

In [67]:
lr = LogisticRegression()
lr.fit(X, y)

print('Logreg intercept:', lr.intercept_)
print('Logreg coef(s):', lr.coef_)
print('Logreg predicted probabilities:', lr.predict_proba(X[0:5,:]))

Logreg intercept: [ 2.29679909 -2.63650072 -3.71046044]
Logreg coef(s): [[-0.1096753   0.1179682   0.11222695 ..., -0.70791925  0.25322048
   0.72242858]
 [-0.14108511 -0.23570642 -0.2716486  ...,  0.99141145  0.00489344
  -0.45665065]
 [ 0.40972848  0.0263229   0.2050937  ..., -0.24414754 -0.30774031
  -0.41885802]]
Logreg predicted probabilities: [[ 0.9544442   0.02200659  0.02354921]
 [ 0.9529399   0.03619128  0.01086882]
 [ 0.87380439  0.06853616  0.05765945]
 [ 0.92790362  0.04227912  0.02981726]
 [ 0.85095723  0.12496628  0.02407649]]


In [70]:
np.logspace(-3,3,30)

array([  1.00000000e-03,   1.61026203e-03,   2.59294380e-03,
         4.17531894e-03,   6.72335754e-03,   1.08263673e-02,
         1.74332882e-02,   2.80721620e-02,   4.52035366e-02,
         7.27895384e-02,   1.17210230e-01,   1.88739182e-01,
         3.03919538e-01,   4.89390092e-01,   7.88046282e-01,
         1.26896100e+00,   2.04335972e+00,   3.29034456e+00,
         5.29831691e+00,   8.53167852e+00,   1.37382380e+01,
         2.21221629e+01,   3.56224789e+01,   5.73615251e+01,
         9.23670857e+01,   1.48735211e+02,   2.39502662e+02,
         3.85662042e+02,   6.21016942e+02,   1.00000000e+03])

In [72]:
params = {
    'penalty':['l2','l1'],
    'C':np.logspace(-3,3,30)
}

In [73]:
gs_lr = GridSearchCV(LogisticRegression(random_state=42, n_jobs=-1), param_grid=params, n_jobs=-1,cv=StratifiedShuffleSplit(random_state=42))

In [79]:
start = datetime.now()

gs_lr.fit(X_train,y_train)

end = datetime.now()
print(end - start)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=0.1,
            train_size=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-03,   1.61026e-03,   2.59294e-03,   4.17532e-03,
         6.72336e-03,   1.08264e-02,   1.74333e-02,   2.80722e-02,
         4.52035e-02,   7.27895e-02,   1.17210e-01,   1.88739e-01,
         3.03920e-01,   4.89390e-01,   7.88046e-01,   1.26896e+00,
         2.0433...5e+02,   2.39503e+02,   3.85662e+02,
         6.21017e+02,   1.00000e+03]), 'penalty': ['l2', 'l1']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [80]:
gs_lr.best_estimator_

LogisticRegression(C=1.2689610031679222, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [ ]:
gs_lr_lr.best_estimator_.coef_

In [81]:
results = pd.DataFrame(gs_lr.cv_results_)
results[results['rank_test_score'] < 5].T

,30,31,32,34
mean_fit_time,0.45732,0.56292,0.512088,0.584101
mean_score_time,0.000864768,0.000838161,0.000855446,0.000875425
mean_test_score,0.898346,0.898346,0.898309,0.898309
mean_train_score,0.898444,0.898975,0.89893,0.899649
param_C,1.26896,1.26896,2.04336,3.29034
param_penalty,l2,l1,l2,l2
params,"{'C': 1.26896100317, 'penalty': 'l2'}","{'C': 1.26896100317, 'penalty': 'l1'}","{'C': 2.04335971786, 'penalty': 'l2'}","{'C': 3.29034456231, 'penalty': 'l2'}"
rank_test_score,1,1,3,3
split0_test_score,0.898364,0.897997,0.898364,0.898364
split0_train_score,0.898536,0.899128,0.899046,0.89972


In [82]:
print('score:', gs_lr.best_estimator_.score(X, y))
print('predict_proba:', gs_lr.best_estimator_.predict_proba(X))

score: 0.898979501809
predict_proba: [[ 0.95657806  0.02225322  0.02116871]
 [ 0.96692168  0.02142833  0.01165   ]
 [ 0.86838372  0.05792451  0.07369177]
 ..., 
 [ 0.889696    0.08866285  0.02164115]
 [ 0.92811989  0.0269709   0.04490921]
 [ 0.95176279  0.02382861  0.0244086 ]]


In [83]:
from sklearn.metrics import classification_report, confusion_matrix

In [86]:
y_test_pred = gs_lr.predict(X_test)

In [87]:
confusion_matrix(y_test, y_test_pred)

array([[12247,     4,     0],
       [  962,     4,     0],
       [  385,     0,     0]])

In [88]:
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       0.90      1.00      0.95     12251
          1       0.50      0.00      0.01       966
          2       0.00      0.00      0.00       385

avg / total       0.85      0.90      0.85     13602



/opt/conda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [92]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import LogisticRegression

In [93]:
param_dict = {
    'mnb':{
        'alpha':np.linspace(.1,1,5)
    },
    'lr':{
        'C':np.logspace(-3,3,7)
    },
    'rf':{}
}

In [94]:
model_dict = {
    'mnb':GridSearchCV(MultinomialNB(),
                             param_grid=param_dict['mnb'],
                             cv=StratifiedShuffleSplit(n_splits=5, random_state=42)),
    'lr':GridSearchCV(LogisticRegression(),
                             param_grid=param_dict['lr'],
                             cv=StratifiedShuffleSplit(n_splits=5, random_state=42)),
    'rf':GridSearchCV(RandomForestClassifier(),
                      param_grid=param_dict['rf'],
                      cv=StratifiedShuffleSplit(n_splits=5, random_state=42)),
}

In [101]:
def fit_all_models(X,y, model_dict):
    for model in model_dict.keys():
        model_dict[model].fit(X,y)
        print("{:5} best score: {}".format(model, model_dict[model].best_score_))

In [102]:
fit_all_models(X_train, y_train, model_dict)

lr    best score: 0.8982539974269436
mnb   best score: 0.8983642712736629
rf    best score: 0.8863811799301599
